In [12]:
import numpy as np
import pandas as pd

gold_matrix = np.array([[1/4, 1/4, 1/2], [1/4, 1/2, 1/4], [1/2, 1/4, 1/4]])
e = np.linalg.eig(gold_matrix)
print(gold_matrix)
print(e)

v = np.zeros(3)
v[0] = 1
r = np.linalg.matrix_power(gold_matrix, 100)@v
print(r)

bc_matrix = np.array([[0.5, 0.45, 0.25], [0.25, 0.1, 0.25], [0.25, 0.45, 0.5]])
e2 = np.linalg.eig(bc_matrix)
print(bc_matrix)

# print(e2)

[[0.25 0.25 0.5 ]
 [0.25 0.5  0.25]
 [0.5  0.25 0.25]]
EigResult(eigenvalues=array([ 1.  , -0.25,  0.25]), eigenvectors=array([[-5.77350269e-01, -7.07106781e-01,  4.08248290e-01],
       [-5.77350269e-01,  1.54173561e-17, -8.16496581e-01],
       [-5.77350269e-01,  7.07106781e-01,  4.08248290e-01]]))
[0.33333333 0.33333333 0.33333333]
[[0.5  0.45 0.25]
 [0.25 0.1  0.25]
 [0.25 0.45 0.5 ]]
